In [1]:
import sys
import os.path
sys.path.append(os.path.join('..', 'util'))


In [2]:
import set_compiler
set_compiler.install()

Compiling with /usr/local/bin/clang-omp


In [3]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x1044030d0>)

In [4]:
import numpy as np
import pylab

In [5]:
import filtering
from timer import Timer
import threading
import time

In [6]:
def iplot(image):
    
    pylab.gray()

    pylab.imshow(image[1200:1800, 3000:3500])
    pylab.title(str(image))
    pylab.show()

In [8]:
def numpy_median(image, iterations=10):
    ''' filter using numpy '''
    for i in range(iterations):
        padded = np.pad(image, 1, mode='edge')
        stacked = np.dstack((padded[:-2,  :-2], padded[:-2,  1:-1], padded[:-2,  2:],
                             padded[1:-1, :-2], padded[1:-1, 1:-1], padded[1:-1, 2:],
                             padded[2:,   :-2], padded[2:,   1:-1], padded[2:,   2:]))
        image = np.median(stacked, axis=2)

    return image

In [8]:
def py_median_3x3(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    
    for i in range(iterations):
        filtering.median_3x3(tmpA, tmpB, 0, 1)
        tmpA, tmpB = tmpB, tmpA



#     N = num_threads
#     tmpA2 = image.copy()
#     tmpB2 = np.empty_like(tmpA)
#     for i in range(iterations):  
#         for n in range(num_threads):
#             th = threading.Thread(target=filtering.median_3x3, args=(tmpA2
#                                                                      ,tmpB2, n, N))# 0, 1
#             #skipping along columns using index and step
#             #tmpA2, tmpB2 = tmpB2, tmpA2
#             #th.daemon = True
#             th.start()
#         th.join()
#         tmpA2, tmpB2 = tmpB2, tmpA2
        
        
        
    return tmpA

In [9]:
def py_median_3x3_threading(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    
    N = num_threads
    tmpA2 = image.copy()
    tmpB2 = np.empty_like(tmpA)
    for i in range(iterations):  
        for n in range(num_threads):
            th = threading.Thread(target=filtering.median_3x3, args=(tmpA2
                                                                     ,tmpB2, n, N))# 0, 1
            #skipping along columns using index and step
            #tmpA2, tmpB2 = tmpB2, tmpA2
            #th.daemon = True
            th.start()
        th.join()
        tmpA2, tmpB2 = tmpB2, tmpA2
        
        
        
    return tmpA2

In [9]:
def py_median_3x3_threading2(image, iterations=10, num_threads=8):
    ''' repeatedly filter with a 3x3 median
        using threading
     '''
    
    N = num_threads
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    e = [threading.Event() for i in range(N+1)]
    for n in range(num_threads):
        th = threading.Thread(target=filtering.median_3x3, args=(tmpA, tmpB, n, N))
        th.daemon = True
        th.start()
        #e[n].set()
#         e[n-1].set()
#         e[n+1].set()
        for i in range(iterations): 
#             e[n-1].set()
#             e[n+1].set()
            #th = threading.Thread(target=filtering.median_3x3, args=(tmpA, tmpB, n, N))#+i works well
            #time.sleep(.05)
            #th.start()
#             e[n-1].clear()
#             e[n+1].clear()
        #e[n].set()

#         e[n].clear()
#         e[n-1].clear()
#         e[n+1].clear()
    
        th.join()
        tmpA, tmpB = tmpB, tmpA
        
    return tmpA

IndentationError: expected an indented block (<ipython-input-9-56f5b8d1118e>, line 31)

In [10]:
input_image = np.load('image.npz')['image'].astype(np.float32)
#from_cython = py_median_3x3_threading3(input_image, 5, 2) #original 2 iter

In [91]:
from_numpy1 = numpy_median(input_image, 1)
from_numpy2 = numpy_median(input_image, 2)#original 2 iter
from_numpy3 = numpy_median(input_image, 3)#original 2 iter
from_numpy4 = numpy_median(input_image, 4)#original 2 iter
from_numpy5 = numpy_median(input_image, 5)#original 2 iter

In [15]:
from_cython = py_median_3x3_threading3(input_image, 3, 2) #original 2 iter
from_numpy2 = numpy_median(input_image, 3)#original 2 iter
assert np.all(from_cython == from_numpy2)

AssertionError: 

In [100]:
iplot(from_cython)

In [122]:
print sum(sum(from_cython[:100,:100]==from_numpy2[:100,:100]))

10000


In [ ]:
if __name__ == '__main__':
    input_image = np.load('image.npz')['image'].astype(np.float32)

    pylab.gray()

    pylab.imshow(input_image)
    pylab.title('original image')

    pylab.figure()
    pylab.imshow(input_image[1200:1800, 3000:3500])
    pylab.title('before - zoom')

    # verify correctness
    from_cython = py_median_3x3_threading(input_image, 1, 2) #original 2 iter
    from_numpy = numpy_median(input_image, 1)#original 2 iter
    assert np.all(from_cython == from_numpy)
    
    
    with Timer() as t:
        
        new_image = py_median_3x3_threading(input_image, 10, 8)

    pylab.figure()
    pylab.imshow(new_image[1200:1800, 3000:3500])
    pylab.title('after - zoom')

    print("{} seconds for 10 filter passes.".format(t.interval))
    pylab.show()


In [11]:
def worker(in_coord, out_coord, iterations, threadid, thread_num, lock):
    #this is close but the iteration conditions still cause a boundary issue
    for i in range(iterations): 
        
        if i == 0:
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'g', threadid, i, lock
                lock[threadid] = i
                
        if i > 0 and i < (iterations - 1) and threadid > 0 and threadid < (thread_num - 1):
            if lock[threadid + 1] == (i - 1) and lock[threadid - 1] == (i - 1):
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'g', threadid, i, lock
                lock[threadid] = i
            else:
                time.sleep(.5)
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'b', threadid, i, lock
                lock[threadid] = i  
                
        if i > 0 and threadid == 0:

            if lock[threadid + 1] == (i - 1):
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'g', threadid, i, lock
                lock[threadid] = i
            else:
                time.sleep(.5)
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'b', threadid, i, lock
                lock[threadid] = i 
            
            
        else:
            if lock[threadid - 1] == (i - 1):
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'g', threadid, i, lock
                lock[threadid] = i
            else:
                time.sleep(.5)
                filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
                in_coord, out_coord = out_coord, in_coord
                #print 'b', threadid, i, lock
                lock[threadid] = i
            
            

def py_median_3x3_threading3(image, iterations, num_threads):
    ''' repeatedly filter with a 3x3 median
        using threading
     '''
    th2 = []
    ev = []
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)

    ev = np.array(ev).reshape((num_threads,iterations))
#     ev = np.array([[threading.Event()]*iterations]*num_threads)
    for n in range(num_threads):
        th = threading.Thread(target=worker2, args=(tmpA, tmpB, iterations, n, num_threads, ev))
        th.start()
        th2 += [th]

        
    return tmpA

In [12]:
def worker2(in_coord, out_coord, iterations, threadid, thread_num, event_array):
    for i in range(iterations):
        
#         if thread_num > 1:
#             if i == 0:
#                 pass
#             elif threadid == 0:
#                 event_array[threadid + 1,i - 1].wait()
#             elif threadid == thread_num - 1:
#                 event_array[threadid - 1, i - 1].wait()
#             else:
#                 event_array[threadid - 1, i - 1].wait()
#                 event_array[threadid + 1, i - 1].wait()

        if thread_num > 1:
            if i == 0:
                pass
            elif threadid == 0:
                event_array[threadid + 1,i - 1].wait()
            elif threadid == thread_num - 1:
                event_array[threadid - 1, i - 1].wait()
            else:
                event_array[threadid - 1, i - 1].wait()
                event_array[threadid + 1, i - 1].wait()
                
        filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
        event_array[threadid, i].set() #stops the thread
        in_coord, out_coord = out_coord, in_coord

In [13]:
t = np.array([range(5)]*5).astype(np.float32)
py_median_3x3_threading3(t, 10, 2)

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.]], dtype=float32)

In [256]:
def fun1(j):
    for i in range(len(j)):
        j[i] = 1
    
def fun2():
    t = np.zeros(5)
    th = threading.Thread(target=fun1, args=(t,))
    th.start()
    th.join()
    return t

In [13]:
s = threading.Semaphore(1)

In [14]:
s.acquire()

True

In [19]:
s = [threading.Event()]*4

In [20]:
k = s[0]

In [21]:
k.set()

In [22]:
k.wait()

True

In [23]:
k.wait()

True

In [24]:
k.set()

In [34]:
ev = np.array([[threading.Event()]*5]*5)

In [43]:
np.shape(ev)

(5, 5)